# ¡Pongamos a trabajar las redes neuronales de moda!

## Objetivos
Basándonos en los repositorios de [UKPLab](https://github.com/UKPLab):

- https://github.com/UKPLab/acl2017-neural_end2end_am (Argument minning con Lasagne)
- https://github.com/UKPLab/emnlp2017-bilstm-cnn-crf (Etiquetado de secuencias)


- Introducirnos en las redes neuronales usando el framework *Keras* conociendo sus distintos backends.
- Conocer las diversas capas y sus utilidades.
- Entender la arquitectura del proyecto actual.

Adaptar el código para..
- Utilizar la última version de Keras y TF para poder utilizar CNNs sin necesidad de usar otro backend.
- Desarrollar un entorno para la identificación y etiquetado de argumentos en textos del idioma inglés.
- Proponer alternativas de embeddings distintos para entrenamiento/evaluación.
- Flexibilizar input para formato texto o formato CoNLL.
- Reportar resultados con diversos parámetros.

## ¿De qué se trata el proyecto aparte de aprender a usar Keras y TF?

El objetivo principal de la tarea es el **correcto etiquetado de argumentos en ensayos**. La identificación de argumentos conocido como **Argument Mining** tiene la particularidad de necesitar un **contexto muy amplio**, es decir necesitan recordar durante todo el documento detalles previos para un correcto funcionamiento.

Como muchas otras tareas semánticas del proceso de lenguaje que necesitan un gran contexto se proponen **redes neuronales con memoria**, es decir no simplemente una CNN sino algo que conserve una memoria mas allá de la iteración. Éstas son las llamadas **LSTM**(Long Short-Term Memory). 

Pero si sólo nos quedáramos con esto no aprenderíamos del pasado, es por eso que dicha red deberá ser Bidireccional teniendo así una red **BiLSTM**

### En resumen:
Combinando una red neuronal BiLSTM con diversos ajustes, con otras capas para realizar la tarea de detección de argumentos en ensayos.

## Algo de lo que aprendimos...  ``from keras.layers.import * ``

Breve descripción de cada capa y su utilidad.

- **CNN**(Convolutional Neuronal Network): Intenta buscar patrones similares en el espacio de los datos de entrada. Ideal para la detección de patrones en imágenes, tales como bordes colores, formas, etc. No tiene en cuenta contexto a la hora de detectar un patrón.

- **RNN**(Recurrent Neuronal Network): Similar a una capa convolucional pero orientada a detectar patrones similares en el tiempo, ideal para procesamiento de texto debido a que de esta forma se basa en el contexto para tomar una determinación. Por ejemplo, en el caso de traduccion automática, en el caso de "dog" tendrá en cuenta si es precedida por "hot".

- **LSTM**(Long Short-Term Memory): A diferencia de las demás, estas poseen una memoria que es olvidada en un tiempo arbitrario. De esta forma logran tener mucho mas contexto en las decisiones.

- **Embedding**: Simplemente transforma los datos de entrada en un embedding que será la entrada de la siguiente capa.

- **Timedistributed**: Realiza una partición temporal de los datos de entrada, dandole a la siguiente capa los datos fraccionados por lotes.

## Nuestra Arquitectura:

![image](arch.jpeg)
```
________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
========================================================================================
token_emd_input (InputLayer)    (None, None)         0                                            
_______________________________________________________________________________________
casing_emd_input (InputLayer)   (None, None)         0                                            
_______________________________________________________________________________________
token_emd (Embedding)           (None, None, 300)    52205700    token_emd_input[0][0]            
_______________________________________________________________________________________
casing_emd (Embedding)          (None, None, 8)      64          casing_emd_input[0][0]           
_______________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 308)    0           token_emd[0][0]                  
                                                                 casing_emd[0][0]                 
________________________________________________________________________________________
main_LSTM_1 (Bidirectional)     (None, None, 250)    434000      concatenate_1[0][0]              
________________________________________________________________________________________
softmax_output (TimeDistributed (None, None, 7)      1757        main_LSTM_1[0][0]                
========================================================================================
```

## Parece una tarea sencilla.. Veamos las piedras en el camino..



- ¡**Que difícil es leer codigo ajeno..**!

- **Primer output**: Con un accuracy de 50% y al correr el modelo solo etiqueta MajorClaims... ¿RunModel tiene un bug o la efectividad reportada no es correcta? Ambos..

- **Métricas por clase**: Calculamos en cada epoch de entrenamiento la evolución de todas sus categorias.

```
[Claim:Against]: Prec: 0.000, Rec: 0.000, F1: 0.0000
[O]: Prec: 0.833, Rec: 0.414, F1: 0.5527
[Premise:Support]: Prec: 0.492, Rec: 0.960, F1: 0.6504
[Claim:For]: Prec: 0.000, Rec: 0.000, F1: 0.0000
[Premise:Attack]: Prec: 0.000, Rec: 0.000, F1: 0.0000
[Claim:Support:For]: Prec: 0.000, Rec: 0.000, F1: 0.0000
[MajorClaim]: Prec: 0.000, Rec: 0.000, F1: 0.0000

Max: 0.1719 on dev;
```

- **Archivos de etiquetado preliminares de cada epoch**: Generamos archivos preliminares de cada epoch siempre que la métrica F1 se incremente. ``/tmp/[f1_score]_[test o dev].txt"``

- **EarlyStopping no es buena idea**: Desactivamos la heurística de dejar de entrenar luego de que en 5 epochs no se genere ninguna mejora. El modelo sigue mejorando..

- **Probemos simplificar las etiquetas**: Mediante un script de python simplificamos las etiquetas (Claim, Premise, etc) en un modelo más simple que sólo diferencia 4 etiquetas(am_simplest).

- **Llegó Keras 2**: Upgrade de entorno a las últimas versiones de Keras y TF. Ahora podemos usar CNN con TF!

```
_______________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
=======================================================================================
token_emd (Embedding)           (None, None, 300)    52205700                
_______________________________________________________________________________________
casing_emd (Embedding)          (None, None, 8)      64          
_______________________________________________________________________________________
main_LSTM_1 (Bidirectional)     (None, None, 250)    434000      merge_1[0][0]              
_______________________________________________________________________________________
softmax_output (TimeDistributed (None, None, 7)      1757        main_LSTM_1[0][0]                
=======================================================================================


_______________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
=======================================================================================
token_emd_input (InputLayer)    (None, None)         0                                            
_______________________________________________________________________________________
casing_emd_input (InputLayer)   (None, None)         0                                            
_______________________________________________________________________________________
token_emd (Embedding)           (None, None, 300)    52205700    token_emd_input[0][0]            
_______________________________________________________________________________________
casing_emd (Embedding)          (None, None, 8)      64          casing_emd_input[0][0]           
_______________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 308)    0           token_emd[0][0]                  
                                                                 casing_emd[0][0]                 
_______________________________________________________________________________________
main_LSTM_1 (Bidirectional)     (None, None, 250)    434000      concatenate_1[0][0]              
_______________________________________________________________________________________
softmax_output (TimeDistributed (None, None, 7)      1757        main_LSTM_1[0][0]                
=======================================================================================
```


## Algunos resultados

| am_simplest 	              | f1 promedio(dev) | f1 promedio(test) | epochs model |
|-----------------------------|------------------|-------------------|--------------|
| crf-adam             	      | 0.71             | 0.71              | 34 epochs    |
| softmax-nadam               | 0.72             | 0.73              | 20 epochs    |
| softmax-sgd                 | 0.48             | 0.50           	 | 44 epochs    |
| crf-nadam (levy)            | 0.71             | 0.70        	     | 32 epochs    |
| softmax-nadam (glove 100d)  | 0.69             | 0.70        	     | 34 epochs    |
| softmax-nadam-paragraph     | 0.70             | 0.72        	     | 16 epochs    |
| softmax-nadam-paragraph(cnn)| 0.71             | 0.74        	     | 27 epochs    |
	

| am_full (levy)              | f1 promedio(dev) | f1 promedio(test) | epochs model |
|-----------------------------|------------------|-------------------|--------------|
| charEmbedding(lstm)-crf     | 0.46             | 0.68              | 20 epochs    |
| charEmbedding(cnn)-crf      | 0.48             | 0.46              | 39 epochs    |
| charEmbeddings(cnn)-softmax | 0.46             | 0.71              | 23 epochs    |
| softmax-paragraph(cnn)      | 0.51             | 0.56              | 26 epochs    |
| softmax                     | 0.44             | 0.47              | 49 epochs    |

# Trabajo relacionado

## Input

- Añadir más información de interés en el ingreso a la red. (ngrams, perfeccionar el casing, posición dentro de la oración, etc)

- Entrenar un embedding propio y utilizarlo para la vectorización de palabras.

- Añadir temática o tópico del ensayo como feature.

## Red

- Si bien las funciones de pérdida exigidas en esta red son de tipo **sparse** adaptar para utilizar alguna otra funcion de perdida

- Probar resultados con solo una simple RNN retirando la capa LSTM para justificar una mejora en el uso de una LSTM.

- Añadir Attention en LSTM

## Output

- Permitir distintas salidas configurables por usuario.

## Algunos conceptos de interés
-  [¿Como crear un modelo simple con Keras y TF?](BuildModel.ipynb)
-  [RunModel es un script para correr un modelo pero..¿Que hace el script?](RunModel.ipynb)
-  [Informe final](https://docs.google.com/document/d/1PGZu9Et5rjSjlU7PbYnwPcyE4B0-yQDZkN372czhV5o/)